In [182]:
import pandas as pd
import numpy as np
import datetime as datetime
from datetime import timedelta
from faker import Faker

In [183]:
class NormalUser:
  def __init__(self, User_ID, IP_Address):
    self.User_ID = User_ID
    self.IP_Address = IP_Address
    self.maxDailyFiles = 25
    self.minDailyFiles = 0
    self.startTime = datetime.time(9,0)
    self.endTime = datetime.time(17,0)

In [184]:
class AbnormalUser:
  def __init__(self, User_ID, IP_Address):
    self.User_ID = User_ID
    self.IP_Address = IP_Address
    self.maxDailyFiles = 1000
    self.minDailyFiles = 10
    self.startTime = datetime.time(0,0)
    self.endTime = datetime.time(23,59)

In [185]:
class FileActions:
  def __init__(self, Sequence):
    self.Sequence = Sequence

In [186]:
class FileInfo:
  def __init__(self,file_name,entropy,file_path, in_use, exists):
    self.file_name = file_name
    self.entropy = entropy
    self.file_path = file_path
    self.in_use = in_use
    self.exists = exists

In [187]:
file_actions = [FileActions('Open->Close'),
                FileActions('Open->Modify->Close'),
                #FileActions('Create'),
                #FileActions('Rename'),
                FileActions('Delete'),
                #FileActions('Create->Open')
                ]

In [188]:
file_info = [
    FileInfo("file1.txt", 7, "/home/user/file1.txt", False, True),
    FileInfo("file2.txt", 6, "/home/user/file2.txt", False, True),
    FileInfo("file3.txt", 4, "/home/user/file3.txt", False, True),
    FileInfo("file4.txt", 5, "/home/user/file4.txt", False, True),
    FileInfo("file5.txt", 4.5, "/home/user/file5.txt", False, True),
    FileInfo("file6.txt", 5.5, "/home/user/file6.txt", False, True),
    FileInfo("file7.txt", 6.5, "/home/user/file7.txt", False, True),
    FileInfo("file8.txt", 7.5, "/home/user/file8.txt", False, True),
    FileInfo("file9.txt", 8, "/home/user/file9.txt", False, True),
    FileInfo("file10.txt", 3, "/home/user/file10.txt", False, True)
]

In [189]:
normal_ips = ["205.175.106.235", "45.24.58.0", "172.120.61.61",
              "162.89.0.47", "50.73.157.178"]

In [190]:
abnormal_ips = ["218.107.132.66", "103.11.0.0", "100.43.64.7",
                "82.213.0.0", "103.48.198.141"]

In [191]:
def generate_date():
    start_date = datetime.datetime(2023, 1, 1)
    end_date = datetime.datetime(2023, 3, 1)
    fake = Faker()
    random_datetime = fake.date_between(start_date, end_date)
    return random_datetime

In [192]:
def time_plus(time, timedelta):
    start = datetime.datetime(
        2000, 1, 1,
        hour=time.hour, minute=time.minute, second=time.second)
    end = start + timedelta
    return end.time()

In [193]:
user_data = pd.DataFrame(columns = ['user_id','user_ip','date','time','file','action','entropy'])

In [194]:
ip = normal_ips[0]
user_id = "user1"
nu = NormalUser(user_id,ip)
num_files = np.random.randint(nu.minDailyFiles, nu.maxDailyFiles, 1)[0]
print(num_files)
random_date = generate_date()
random_hour = np.random.randint(nu.startTime.hour, nu.endTime.hour)
random_minute = np.random.randint(0,59)
random_second = np.random.randint(0,59)
random_time =datetime.time(hour=random_hour,minute=random_minute,second=random_second)
#print(random_time)
# print("random_date: ", random_date)
# print("random_time: ", random_time)
flen = len(file_info)
aclen = len(file_actions)
#files = np.random.randint(flen, size=num_files)
new_time = random_time
for i in range(num_files):
    #new_time = random_time
    file_random_number = np.random.randint(flen,size=1)[0]
    ac_random_number = np.random.randint(aclen,size=1)[0]
    #file_obj = file_info[files[i]]
    file_obj = file_info[file_random_number]
    #print(file_obj.file_name)
    ac_obj = file_actions[ac_random_number]
    print(file_obj.file_name)
    if file_obj.exists == False:
        print("Does not exist")
        continue
    
    #new_time = random_time
    if len(ac_obj.Sequence) > 1:
        num_actions = ac_obj.Sequence.split("->")
        #new_time = random_time
        for action in num_actions:
            row = []
            if action == "Open":
                file_obj.in_use = True
            else:
                file_obj.in_use = False
            if action == "Modify":
                change = np.random.uniform(-0.1,0.1)
                file_obj.entropy += change
            if action == "Delete":
                if file_obj.in_use == False and file_obj.exists == True:
                    file_obj.exists = False
                else:
                    break
            random_interval = np.random.randint(1,10)
            time_change = datetime.timedelta(minutes=random_interval)
            new_time = time_plus(new_time, time_change)
            #new_time = new_time + time_change
            row = []
            row.append(user_id)
            row.append(ip)
            row.append(random_date)
            row.append(new_time)
            row.append(file_obj.file_name)
            row.append(action)
            row.append(file_obj.entropy)
            user_data = pd.concat([user_data,pd.DataFrame([row],columns = ['user_id','user_ip','date','time','file','action','entropy'])], ignore_index=True)
            user_data.reset_index()
            
    else:
        action = ac_obj.Sequence
        #new_time = random_time
        if action == "Open":
                file_obj.in_use = True
        else:
                file_obj.in_use = False


        if action == "Delete":
            if file_obj.in_use == False and file_obj.exists == True:
                file_obj.exists = False
            else:
                pass
        elif action == "Modify":
            change = np.random.uniform(-0.1,0.1)
            file_obj.entropy += change
            
        random_interval = np.random.randint(1,10)
        time_change = datetime.timedelta(minutes=random_interval)
        new_time = time_plus(new_time, time_change)
        #new_time = new_time + time_change
        row = []
        row.append(user_id)
        row.append(ip)
        row.append(random_date)
        row.append(new_time)
        row.append(file_obj.file_name)
        row.append(action)
        row.append(file_obj.entropy)
        user_data = pd.concat([user_data,pd.DataFrame([row],columns = ['user_id','user_ip','date','time','file','action','entropy'])], ignore_index=True)
        user_data.reset_index()

22
file4.txt
file4.txt
file3.txt
file1.txt
file8.txt
file3.txt
file8.txt
Does not exist
file3.txt
Does not exist
file3.txt
Does not exist
file7.txt
file4.txt
Does not exist
file2.txt
file6.txt
file7.txt
file3.txt
Does not exist
file4.txt
Does not exist
file8.txt
Does not exist
file2.txt
file10.txt
file1.txt
Does not exist
file10.txt
file9.txt


In [195]:
#user_data.sort_values(by='time',inplace=True)
user_data

,user_id,user_ip,date,time,file,action,entropy
0,user1,205.175.106.235,2023-01-18,10:16:32,file4.txt,Open,5
1,user1,205.175.106.235,2023-01-18,10:19:32,file4.txt,Modify,5.033915
2,user1,205.175.106.235,2023-01-18,10:27:32,file4.txt,Close,5.033915
3,user1,205.175.106.235,2023-01-18,10:29:32,file4.txt,Delete,5.033915
4,user1,205.175.106.235,2023-01-18,10:32:32,file3.txt,Open,4
5,user1,205.175.106.235,2023-01-18,10:34:32,file3.txt,Close,4
6,user1,205.175.106.235,2023-01-18,10:39:32,file1.txt,Delete,7
7,user1,205.175.106.235,2023-01-18,10:44:32,file8.txt,Delete,7.5
8,user1,205.175.106.235,2023-01-18,10:50:32,file3.txt,Delete,4
9,user1,205.175.106.235,2023-01-18,10:51:32,file7.txt,Open,6.5
